<a href="https://colab.research.google.com/github/Surojit-KB/Univ-ai-ml1c1-labs/blob/main/distribute_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 1: Goodreads Parsing and Scraping
**Univ.AI** <br>
**DS-1 Cohort 1**


## Table of Contents
* [Lab1: Goodreads Scraping and Parsing](#Lab1:-Goodreads-Scraping-and-Parsing)
  * [Learning Goals](##Learning-Goals)
  * [Q1: Scrape the "Best Books ever" web page](##Q1:-Scrape-the-"Best-Books-ever"-web-page)
  * [Q2: Parse the page, extract book urls](##Q2:-Parse-the-page,-extract-book-urls)
  * [Q3: Scrape the web page of each book](##Q3:-Scrape-the-web-page-of-each-book)
  * [Q4: Parse each books page, extract information](##Q4:-Parse-each-books-page,-extract-information)
    * [4.1 Extract genres](###4.1-Extract-Genres)
    * [4.2 Extract Published Year](###4.2-Extract-Published-Year)
    * [4.3 Extract Rating, ISBN, Title of the book, Author and Rating Count](###4.3-Extract-Rating,-ISBN,-Title-of-the-book,-Author-and-Rating-Count)
    * [4.4 Creating a DataFrame](###4.4-Creating-a-DataFrame)


## Learning Goals 
Goodreads has put out a list of the "Best Books ever", as voted on by around 200,000 people from the general Goodreads community. 

In this lab, we will be scraping and extracting information from Goodread's "[Best Books ever](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1)" list.

This lab consists of four main parts:
1. Scraping the "Best Books ever" web page
2. Parsing the page, extract book urls
3. Scraping the web page of each book
4. Parsing a book page, extract book properties

This lab will develop your skills in:
* Exploring web pages through developer tools 
* Scraping and Parsing using Beautiful Soup and requests

In [37]:
#Import libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import time, requests

## Q1: Scrape the  "Best Books ever" web page
We're going to see the structure of Goodread's best books list. 

To get this page we use pythons [requests module](https://requests.readthedocs.io/en/master/). 

In [38]:
#Getting the url using requests module
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
page = requests.get(url)

Check the status of the page - 200 is OK and 404 is not good!

In [39]:
page.status_code

200

Now that we are familiar with pythons request module, let us access the first two pages from Goodreads. 

This will mean you will need to scrape **two** URLs: https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 and  https://www.goodreads.com/list/show/1.Best_Books_Ever?page=2. 

<br>

**Hint:** To do this, you can put your request.get() function in a `for` loop.<br>
You will also need to use the time.sleep() function to wait for 1 second between the two get requests so that Goodread's doesn't think you are a threat attempting to mount a denial-of-service attack!<br>
In addition to this, store the HTML text in a dictionary `page_dict`. The key should be the page number (1 or 2) and the value should be the HTML text corresponding the page.

In [78]:
#Loop to fetch 2 pages of "best-books" from goodreads. 
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="

page_dict={}

#your code here
for i in range(1,3):
  url = URLSTART+BESTBOOKS+str(i)
  page = requests.get(url)
  page_dict[i]=page
  time.sleep(5)

## Q2: Parse the page, extract book urls

Our next step should be to parse the HTML text we have saved from the previous section and extract the information we need. 

To do this, we will be using BeautifulSoup to transform HTML content into Python data structures. You can also use other libraries like PyQuery if you are comfortable with jQuery, but we will be using BeautifulSoup.


In [42]:
#Import BeautifulSoup
from bs4 import BeautifulSoup

Our aim is to extract the **book URLs** on the page in order to use it in further sections. 

To do this, we look for the elements with class bookTitle, extract the urls, and write them into a dictionary `urldict` where the keys are the page numbers and the values are a list of the book URLs extracted from the page.


**Hint:** While parsing the HTML, look for the HTML a element, but only the one that has a CSS class of `bookTitle`. If you look at the page source, you'll see a construct like **`class=bookTitle`** on the table as seen below:
<br><br>
![goodreadsexample](https://drive.google.com/uc?export=view&id=1PUvIe7VkXFSkvrj4pZm0pRKWr6W8Rw0F)


In [75]:
urldict={}
titledict={}
#your code here
for i in range(1,3):
  soup=BeautifulSoup(page_dict[i].text,'html.parser')
  urldict[i]=[s['href'] for s in soup.find_all('a','bookTitle')]
  titledict[i]=[s.contents[0] for s in soup.find_all('span',attrs={"itemprop": "name","role":"heading"})]

## Q3: Scrape the web page of each book

Now that we have the book URLs in a dictionary `urldict`, we can parse the web pages of the books itself to extract some information. <br>

Before we extract information, we will need to get the HTML text for each book's page. 
Scrape the books web pages and store the HTML text for each book in a dictionary named `bookdict` using a `for` loop in a similar fashion as before.

In [67]:
#This is just an **example** to understand how to scarpe these files
#Scraping one of the files
URLSTART="https://www.goodreads.com"
book_url=URLSTART+urldict[2][0]
stuff=requests.get(book_url)

#Check the status of the page
print(stuff.status_code)

#All OK!

200


##Q4: Parse each books page, extract information

Now that we have the HTML text for the books, we can extract information from these web pages. 
We intend to extract the following data:

- Published year
- Rating
- ISBN 
- Title of the book
- Author
- Genres this book fits in. Since there are several genres associated with each book, you will need to extract the URL of each genre, separated by a pipe '|' like so:
```
/genres/young-adult|/genres/fiction|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|
```
- Rating count, the number of people who have rated this book
<br>
<br> 
All this information can be seen on the web page.
You will need to go to the developer tools and extract the necessary information. 
<br>
<br>

Since Published year and Genre require some extra processing to be extracted, we will start by writing 2 functions - `get_genre` and `get_year`.

### 4.1 Extract Genres

Write a function to get the genres which takes as input the HTML text and outputs a list of the genre URLs.

In [45]:
#Extracting genre
#your code here
def get_genre(d):
  genre="|".join([g['href'] for g in d.find_all("a",attrs={"class": "actionLinkLite bookPageGenreLink"})])
  return genre

### 4.2 Extract Published Year

Write a function to get the published year which takes as input the HTML text.

You might have to use regular expressions to extract only the year Published Date seen in the web pages.

**Regular expressions** <br>
Regular Expressions is a pattern matching mechanism used throughout Computer Science and programming (it's not just specific to Python). A tutorial on Regular Expressions (aka regex) is beond this lab, but below are many great resources that we recommend, if you are interested in them (could be very useful for a homework problem):<br>
https://docs.python.org/3.3/library/re.html <br>
https://regexone.com <br>
https://docs.python.org/3/howto/regex.html <br>


In [154]:
#Extracting published year
yearre = r'\d{4}'
def get_year(d):
  years=d.find("div", attrs={"class": "uitext darkGreyText"})
  years=years.findChildren("div")[1].text
  yearmatch=re.findall(yearre,years)
  years_original=d.find_all("nobr", attrs={"class": "greyText"})
  if years_original!=[]:
    finalyear=years.strip()[-5:-1]
    return finalyear
  else:
      if len(yearmatch) > 0:
          finalyear=yearmatch[0]
      else:
          finalyear="NA"
      return finalyear

### 4.3 Extract Rating, ISBN, Title of the book, Author and Rating Count

Now that you have created functions to extract genres and published years, you can extract the rest of the fields in a line or two.

Extract the other fields and incorporate your functions to get a **list of dictionaries**. Each element in the list is a dictionary with the information you have extracted (Published year, Rating, ISBN, Title of the book, Author, Genres, Rating Count).
<br>
<br>
So **each element in the list** should look something like this:
```
{'author': 'https://www.goodreads.com/author/show/153394.Suzanne_Collins',
 'booktype': 'books.book',
 'rating': 4.33,
 'genres': '/genres/young-adult|/genres/fiction|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action',
 'isbn': '9780439023481',
 'ratingCount': '6554254',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'year': '2008'}
```

**Note**: Remember to convert your list of genres to a string seperated with the pipe character '|'.


In [187]:
#your code here
def get_details(d,i,j):
  try:
    isbn=d.find("span",attrs={"itemprop": "isbn"}).contents[0]
  except AttributeError:
    isbn='NA'
  rating=float(d.find("span",attrs={"itemprop": "ratingValue"}).contents[0].strip())
  ratingCount=int(d.find("meta",attrs={"itemprop": "ratingCount"})['content'])
  author_url=",".join([author['href'] for author in book_soup.find("span",attrs={"itemprop": "author"}).findChildren('a')])
  author_name=",".join([soup.contents[0] for soup in book_soup.find_all("span",attrs={"itemprop": "name"})])
  title=titledict[i][j]
  year=int(get_year(d))
  genre=get_genre(d)
  detail_dict={'author_url':author_url,'author_name':author_name,'rating':rating,'genres':genre,'isbn':isbn,'ratingCount':ratingCount,'title':title,'year':year}
  return detail_dict

In [188]:
from tqdm.notebook import tqdm

In [189]:
#Fetching the actual 200 book pages
#In the interest of time, we are taking just the first 10 of each page. Running this for 200 books takes 25 min!
bookdict={}
listofdicts=[]
#your code here
for i in range(1,3):
  for j in tqdm(range(100)):
    url=URLSTART+urldict[i][j]
    book_data=requests.get(url).text
    book_soup=BeautifulSoup(book_data,'html.parser')
    bookdict[i]=book_soup
    listofdicts.append(get_details(book_soup,i,j))
    time.sleep(10)


### 4.4 Creating a DataFrame

Convert the list of dictionaries created above to a Pandas DataFrame

In [190]:
df = pd.DataFrame.from_records(listofdicts)
df.iloc[10:20]

,author_url,author_name,rating,genres,isbn,ratingCount,title,year
10,https://www.goodreads.com/author/show/11081.Ma...,Margaret Mitchell,4.30,/genres/classics|/genres/historical|/genres/hi...,9780446675536,1093201,Gone with the Wind,1936
11,https://www.goodreads.com/author/show/4.Dougla...,Douglas Adams,4.22,/genres/science-fiction|/genres/fiction|/genre...,NA,1485735,The Hitchhiker's Guide to the Galaxy (Hitchhik...,1979
12,https://www.goodreads.com/author/show/435477.S...,Shel Silverstein,4.37,/genres/childrens|/genres/childrens|/genres/pi...,9780060256654,932836,The Giving Tree,1964
13,https://www.goodreads.com/author/show/4191.Emi...,"Emily Brontë,Richard J. Dunn,Charlotte Brontë,...",3.86,/genres/classics|/genres/fiction|/genres/roman...,9780393978896,1384781,Wuthering Heights,1847
14,https://www.goodreads.com/author/show/630.Dan_...,Dan Brown,3.86,/genres/fiction|/genres/mystery|/genres/thrill...,NA,1984297,"The Da Vinci Code (Robert Langdon, #2)",2003
15,https://www.goodreads.com/author/show/3565.Osc...,"Oscar Wilde,Jeffrey Eugenides",4.09,/genres/classics|/genres/fiction|/genres/horro...,NA,1007107,The Picture of Dorian Gray,1890
16,https://www.goodreads.com/author/show/614.Arth...,Arthur Golden,4.12,/genres/fiction|/genres/historical|/genres/his...,9781400096893,1752039,Memoirs of a Geisha,1997
17,https://www.goodreads.com/author/show/1036615....,"Charlotte Brontë,Michael Mason,Sylvère Monod,B...",4.13,/genres/classics|/genres/fiction|/genres/roman...,9780142437209,1672342,Jane Eyre,1847
18,https://www.goodreads.com/author/show/8164.Lew...,"Lewis Carroll,John Tenniel,Martin Gardner",4.06,/genres/classics|/genres/fantasy|/genres/ficti...,9780451527745,472610,Alice's Adventures in Wonderland / Through the...,1871
19,https://www.goodreads.com/author/show/13661.Vi...,"Victor Hugo,Lee Fahnestock,Norman MacAfee",4.18,/genres/classics|/genres/fiction|/genres/histo...,NA,695903,Les Misérables,1862


Convert this dataframe to a csv file and store it using `to_csv`.

In [191]:
df.to_csv("Goodreads.csv", index=False, header=True)